#Instalação das bibliotecas

In [ ]:
!pip install pandas numpy chromadb openai

!pip install -U langchain langchain-community langchain-chroma langchain-openai langchain-experimental

!pip install --upgrade langchain

!pip install --upgrade langchain openai

!pip install --upgrade openai

!pip install nltk rouge-score

In [48]:
!pip install -U spacy
!python -m spacy download pt_core_news_sm

!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 54.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

#Importação das bibliotecas

In [ ]:
import openai
import numpy as np
import pandas as pd
import os
import sys
import langchain
import re
import time

from langchain_openai import OpenAIEmbeddings  #API do GPT - Constrói pipelines de IA com memória, contexto, prompts, integrações
from langchain.schema import Document
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA       #Cria pipeline RAG (busca contexto + gera resposta)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough, RunnableMap, RunnableLambda
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_chroma import Chroma            #Banco vetorial
from langchain.chains.combine_documents.refine import RefineDocumentsChain
from langchain.docstore.document import Document
from langchain.chains.qa_with_sources.base import QAWithSourcesChain
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from openai import RateLimitError
from openai import OpenAI
from google.colab import drive
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

In [49]:
import spacy
from bert_score import score

nlp = spacy.load("pt_core_news_sm")

#OpenAI API Key

In [ ]:
from google.colab import userdata
api_key=userdata.get('OPENAI_API_KEY')
print(f"🔑 Chave de API: {api_key[:4]}...{api_key[-4:]}")

🔑 Chave de API: sk-p...3aYA


#Carregamento do LLM

In [ ]:
#Leitura do Excel e conversão em documentos do LangChain
filepath = "Dicionario_Vacina_RNA.xlsx"

def load_excel_as_documents(filepath):
    sheets = pd.read_excel(filepath, sheet_name=None)
    documents = []
    for sheet_name, df in sheets.items():
        df = df.dropna(how="all")
        for _, row in df.iterrows():
            content = " | ".join([str(cell) for cell in row if isinstance(cell, str)])
            if content.strip():
                documents.append(Document(
                    page_content=f"[{sheet_name}] {content}",
                    metadata={"sheet": sheet_name}
                ))
    return documents

# Carrega os dados do Excel como documentos
documents = load_excel_as_documents(filepath)

#Divisão do texto em blocos, utilizando Langchain

In [ ]:
#Divide em blocos de até 500 caracteres com sobreposição de 50 caracteres por bloco
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", ",", " ", ""]   #Splitter q prioriza sentenças
)
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

#Geração dos embeddings, utilizando OpenAI

In [ ]:
#Gerar embeddings (vetores numéricos) densos para cada bloco gerado
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=api_key)

<ipython-input-10-10640e635b7c>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=api_key)


#Criação e persistência do banco vetorial ChromaDB

In [ ]:
#Criar pasta para salvar o Chroma DB no Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Diretório onde será salvo o Chroma DB
db_path = "/content/drive/My Drive/banco_Chroma/chroma_RNA"

#Criar ou carregar o banco vetorial (vectorstore) com Chroma
if os.path.exists(db_path) and os.listdir(db_path):
    print("Banco vetorial já existe. Carregando...")
    vectorstore = Chroma(
        persist_directory=db_path,
        embedding_function=embedding_model
    )
else:
    print("Banco vetorial não encontrado. Criando novo...")
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        persist_directory=db_path
    )
    print("Banco vetorial criado e salvo com sucesso!")

Banco vetorial não encontrado. Criando novo...
Banco vetorial criado e salvo com sucesso!


#Configuração do modelo conversacional (LLM OpenAI)

In [ ]:
#Cria o modelo conversacional
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=300, openai_api_key=api_key)

#MODELO INICIAL

##Criação da cadeia RAG com LangChain usando PromptTemplate

###PromptTemplate

In [638]:
qa_prompt = PromptTemplate.from_template(
"""
Você é um assistente especialista em dados de planilhas. Com base no {context} do Excel, responda com precisão à pergunta: {question}
Responda de forma clara, objetiva e com linguagem técnica acessível.
"""
)

###Cadeia de documentos com Runnable Passthrough

In [639]:
# Cria a cadeia de processamento dos documentos
document_chain = RunnablePassthrough() | qa_prompt | llm

###Recuperador

In [640]:
# Buscar documentos com score e filtrar por limiar
def retrieve_with_threshold_fn(pergunta, threshold=0.7):
    # Recupera documentos com seus scores
    resposta_obtida = vectorstore.similarity_search_with_score(pergunta, k=10)
    # Filtra por similaridade mínima
    filtered_docs = [doc for doc, score in resposta_obtida if score >= threshold]
    # Se nada passou do limiar, retorna os top-3 independente do score
    if not filtered_docs:
       filtered_docs = [doc for doc, _ in resposta_obtida[:3]]

    return filtered_docs[:5]  # Retorna os 5 melhores após filtro

###Fluxo de RAG usando RunnablePassthrought

In [641]:
# Função inteligente para juntar contexto (para debug, se necessário)
def juntar_contexto(docs):
    if not docs:
        raise ValueError("⚠️ Nenhum contexto relevante foi encontrado para essa pergunta.")

    first = docs[0]
    if isinstance(first, str):
        return "\n\n".join(docs)
    elif hasattr(first, "page_content"):
        return "\n\n".join(doc.page_content for doc in docs)
    else:
        raise ValueError(f"❌ Formato desconhecido: {type(first)}")

In [642]:
# Criação da cadeia de recuperação: Faz a pergunta ➔ busca documentos ➔ junta contexto ➔ consulta o modelo.
retrieval_chain = RunnablePassthrough().assign(
    question=lambda x: x["question"]
).assign(
    documents=lambda inputs: retrieve_with_threshold_fn(inputs["question"], threshold=0.7)
).assign(
    context=lambda inputs: juntar_contexto(inputs["documents"])
).assign(
    answer=lambda inputs: document_chain.invoke({
        "context": inputs["context"],
        "question": inputs["question"]
    })
)

###Chatbot interativo

In [643]:
#Função RAG que traz as respostas obtidas pelo modelo:
def rag_with_context(pergunta: str):
    max_retries = 3
    attempt = 0

    while attempt < max_retries:
        try:
            resposta_obtida = retrieval_chain.invoke({"question": pergunta}) #LangChain que recupera e gera texto.

            # Tratamento das respostas obtidas e conteúdo
            if isinstance(resposta_obtida, dict):
                conteudo = resposta_obtida.get("answer", resposta_obtida.get("output", resposta_obtida))
            else:
                conteudo = resposta_obtida

            if hasattr(conteudo, "content"):
                resposta_texto = conteudo.content
            elif isinstance(conteudo, str):
                resposta_texto = conteudo
            else:
                resposta_texto = str(conteudo)

            # Contexto
            contexto_usado = resposta_obtida.get("context", "⚠️ Contexto não encontrado")
            return resposta_texto, contexto_usado

#resposta_obtida = resposta original, que pode vir em vários formatos.
#resposta_texto = texto limpo e padronizado, em formato strm pronto para ser retornado.

        except RateLimitError:
            attempt += 1
            if attempt < max_retries:
                print(f"🚫 Limite de requisições atingido. Tentando novamente em 10 segundos... (tentativa {attempt}/{max_retries})")
                time.sleep(10)
            else:
                print("❌ Limite de tentativas atingido. Tente novamente mais tarde.")
                return "", ""
        except Exception as e:
            print(f"⚠️ Erro inesperado: {e}")
            return "", ""

In [644]:
#Loop de perguntas

print("\n✅ Planilha carregada! Faça suas perguntas sobre os dados.")

while True:      #Para inserir perguntas manualmente
    query = input("\n❓ Digite sua pergunta ou 'sair': ")
    if query.lower() in ["sair", "exit", "quit"]:
        print("👋 Encerrando...")
        break

    resposta_texto, contexto_usado = rag_with_context(query)

    if resposta_texto:
        print(f"\n 🤖 Resposta do bot:\n{resposta_texto}")


✅ Planilha carregada! Faça suas perguntas sobre os dados.

❓ Digite sua pergunta ou 'sair': Dê um exemplo de lipídio ionizável

 🤖 Resposta do bot:
Um exemplo de lipídio ionizável é o SM-102, que possui uma carga relativamente neutra em pH fisiológico, mas se torna positivamente carregado dentro da nanopartícula devido à protonação do grupo amina para formar um cátion de amônio. Isso permite que ele se ligue à espinha dorsal negativamente carregada do mRNA.

❓ Digite sua pergunta ou 'sair': Em qual vacina o SM-102 é usado

 🤖 Resposta do bot:
O SM-102 é usado na vacina da Moderna contra a COVID-19. Ele é um amino lipídio sintético que faz parte do sistema de entrega de medicamentos dessa vacina, que é baseada em mRNA.

❓ Digite sua pergunta ou 'sair': Qual vacina contém N1-metilpseudourina

 🤖 Resposta do bot:
A vacina que contém N1-metilpseudourina é a tozinameran (Pfizer-BioNTech) e a elasomeran (Moderna), que são vacinas de mRNA contra o SARS-CoV-2. A N1-metilpseudourina é utilizad

##Métricas da qualidade das respostas e do modelo

#### a) Aplicação das métricas

In [1025]:
#Normalização do texto:
def normalize(resposta_texto):
    resposta_texto = resposta_texto.lower()  # Deixa tudo minúsculo
    resposta_texto = re.sub(r'[^\w\s]', '', resposta_texto)  # Remove pontuação
    resposta_texto = re.sub(r'\s+', ' ', resposta_texto) # Substitui múltiplos espaços por um único espaço
    return resposta_texto.strip()  # Remove espaços nas bordas

In [1026]:
def extract_entities(resposta_texto):
    doc = nlp(resposta_texto)
    return {ent.text.lower() for ent in doc.ents}

In [1027]:
#BLEU (Bilingual Evaluation Understudy)
#Mede precisão = quantos n-gramas da resposta gerada aparecem na referência.
def calc_bleu(resposta_texto, resposta_esperada):
    smooth = SmoothingFunction().method4
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        ref_tokens = [ref.strip().split()]  # BLEU espera lista de listas
        pred_tokens = pred.strip().split()
        score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smooth)
        scores.append(score)

    return np.mean(scores)

In [1028]:
# ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
#Mede recall = quanto da referência aparece na resposta
def calc_rouge(resposta_texto, resposta_esperada):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        score = scorer.score(ref.strip(), pred.strip())
        scores.append(score['rougeL'].fmeasure)

    return np.mean(scores)

In [1029]:
# Coverage Score (quanto da resposta esperada estava na resposta gerada)
def calc_coverage(resposta_texto, resposta_esperada):
    ref_tokens = normalize(resposta_esperada).split()
    pred_tokens = normalize(resposta_texto).split()
    common = set(ref_tokens) & set(pred_tokens)
    if not common:
        return 0.0
    return len(common) / len(ref_tokens)

In [1030]:
# Partial Match (modelo aceito se cobrir pelo menos 50% da resposta)
def calc_partial_match(resposta_texto, resposta_esperada, threshold=0.5):
    coverage = calc_coverage(resposta_texto, resposta_esperada)
    return int(coverage >= threshold)

In [1031]:
#Taxa de alucinação = Percentual de respostas inventadas ou que fugiram do contexto
def check_hallucination(resposta_texto, contexto_usado):
    if "nenhuma informação suficiente" in resposta_texto_1.lower():
        return False  # isso NÃO é alucinação, é fallback correto

    resposta_entidades = extract_entities(resposta_texto)
    contexto_entidades = extract_entities(contexto_usado)

    # Verifica se todas as entidades na resposta estão presentes no contexto
    return not resposta_entidades.issubset(contexto_entidades)

In [1032]:
def calc_bertscore(resposta_texto, resposta_esperada):
    P, R, F1 = score([resposta_texto], [resposta_esperada], lang='pt', verbose=False)
    return F1.item()

####b) Avaliação das respostas obtidas

In [1033]:
# Conjunto de perguntas e respostas esperadas
avaliacao = [
     {"pergunta": "Dê um exemplo de lipídio ionizável", "resposta_esperada": "Um exemplo de lipídio ionizável é o SM-102"},
     {"pergunta": "Em qual vacina o SM-102 é usado", "resposta_esperada":"O SM-102 é usado na vacina da Moderna COVID-19"},
     {"pergunta": "Qual vacina contém N1-metilpseudourina", "resposta_esperada": "A vacina que contém N1-metilpseudouridina é a tozinameran (Pfizer-BioNTech) e elasomeran (Moderna)."},
     {"pergunta": "Qual a função da LNP?", "resposta_esperada": "A função da LNP é atuar como veículo de entrega para o RNA, protegendo-o da degradação e facilitando sua entrada nas células."},
     {"pergunta": "Como é formada uma LNP?", "resposta_esperada": "A LNP é formada por lipídios ionizáveis, fosfolipídios, colesterol e é revestida por lipídios peguilados"},
     {"pergunta": "Dê exemplo de vacina com saRNA", "resposta_esperada": "Um exemplo de vacina com saRNA é Gemcovac, autorizada na Índia em 2022"}
]

In [1034]:
print("\n🔍 Avaliação das Respostas:")
print("=" * 80)

#Inicialização das métricas globais
total_bleu, total_rouge, total_coverage, total_partial, total_bertscore = 0, 0, 0, 0, 0
hallucinated = 0

#Avaliação automática de respostas = compara as respostas obtidas x esperadas
for exemplo in avaliacao:
    pergunta = exemplo["pergunta"]
    resposta_esperada = exemplo["resposta_esperada"]
    try:
        resposta_texto, contexto_usado = rag_with_context(pergunta)
    except Exception as e:
        print(f"❌ Erro ao buscar resposta para '{pergunta}': {e}")
        resposta_texto, contexto_usado = "", ""

#Calcula as métricas da resposta
    bleu = calc_bleu(resposta_texto, resposta_esperada)
    rouge = calc_rouge(resposta_texto, resposta_esperada)
    coverage = calc_coverage(resposta_texto, resposta_esperada)
    partial = calc_partial_match(resposta_texto, resposta_esperada, threshold=0.5)
    aluc = check_hallucination(resposta_texto, contexto_usado)
    bertscore = calc_bertscore(resposta_texto, resposta_esperada)

#Acumula os resultados nas variáveis globais
    total_bleu += bleu
    total_rouge += rouge
    total_coverage += coverage
    total_partial += partial
    total_bertscore += bertscore
    hallucinated += int(aluc)

# Exibe avaliação por pergunta
    print(f"Pergunta: {pergunta}")
    print(f"Resposta esperada: {resposta_esperada}")
    print(f"Resposta do modelo: {resposta_texto}")
    print(f"Contexto: {contexto_usado[:200]}...")  # Mostra só o início
    print(f"bleu={bleu:.4f} | rouge={rouge:.4f} | Coverage={coverage:.2f} | Partial={partial} | BERTScore={bertscore:.4f} | Alucinação={'Sim' if aluc else 'Não'}")
    print("-" * 80)


🔍 Avaliação das Respostas:
Pergunta: Dê um exemplo de lipídio ionizável
Resposta esperada: Um exemplo de lipídio ionizável é o SM-102
Resposta do modelo: Um exemplo de lipídio ionizável é o SM-102.
Contexto: [Planilha2] Ionisable lipids like SM-102 hold a relatively (/ close to) neutral charge at physiological pH but are positively charged within the nanoparticle (the amine group is protonated to form an ...
bleu=0.8333 | rouge=0.7381 | Coverage=1.00 | Partial=1 | BERTScore=0.9823 | Alucinação=Não
--------------------------------------------------------------------------------
Pergunta: Em qual vacina o SM-102 é usado
Resposta esperada: O SM-102 é usado na vacina da Moderna COVID-19
Resposta do modelo: O SM-102 é usado na vacina da Moderna contra a COVID-19.
Contexto: [Planilha2] SM-102 is a synthetic amino lipid which is used in combination with other lipids to form lipid nanoparticles.[1] These are used for the delivery of mRNA-based vaccines,[2][3][4] and in par...
bleu=0.6522 | ro

#### b) Avaliação do modelo

#####***CRITÉRIOS***
#####**Bleu:** O modelo é considerado bom se o Bleu for superior a 0.3.
#####**Rouge:** O modelo é considerado bom se o Rouge for superior a 0.3.
#####**Coverage:** O modelo é considerado bom se o Coverage médio for superior a 0.7.
#####**Partial Match:** O modelo é considerado bom se a taxa de Partial Match for superior a 0.5.
#####**BERTScore:** O modelo é considerado bom se o BERTScore for superior a 0.8.
#####**Alucinação:** O modelo é considerado bom se a taxa de alucinação for menor que 0.7.

#####***AVALIAÇÃO***

In [1036]:
n = len(avaliacao)

# Função e critérios para avaliar se o modelo é bom, com base nas métricas finais
def avaliar_modelo(total_bleu, total_rouge, total_coverage, total_partial, total_bertscore, hallucinated, n):
    bleu_threshold = 0.3
    rouge_threshold = 0.3
    coverage_threshold = 0.7
    partial_threshold = 0.5
    bertscore_threshold = 0.8
    hallucination_threshold = 0.7

    # Calculando as métricas médias
    avg_bleu = total_bleu / n
    avg_rouge = total_rouge / n
    avg_coverage = total_coverage / n
    avg_partial = total_partial / n
    avg_bertscore = total_bertscore / n
    avg_hallucinated = hallucinated / n

    # Exibindo as métricas finais
    print("\nMÉTRICAS FINAIS - Modelo inicial:")
    print(f"Bleu:                {avg_bleu:.4f}")
    print(f"Rouge:               {avg_rouge:.4f}")
    print(f"Coverage médio:      {avg_coverage:.2f}")
    print(f"Partial Match taxa:  {avg_partial:.2f}")
    print(f"BERTScore médio:     {avg_bertscore:.4f}")
    print(f"Alucinação:          {avg_hallucinated:.2f}")

    # Avaliando o modelo com base nas métricas
    if avg_bleu >= bleu_threshold and avg_rouge >= rouge_threshold and avg_coverage >= coverage_threshold and \
       avg_partial >= partial_threshold and avg_bertscore >= bertscore_threshold and avg_hallucinated <= hallucination_threshold:
        print("\n🔍 O modelo é bom!")
    else:
        print("\n⚠️ O modelo precisa de melhorias.")

#Chamada da função para avaliar o modelo
avaliar_modelo(total_bleu, total_rouge, total_coverage, total_partial, total_bertscore, hallucinated, n)


MÉTRICAS FINAIS - Modelo inicial:
Bleu:                0.5435
Rouge:               0.5035
Coverage médio:      0.87
Partial Match taxa:  1.00
BERTScore médio:     0.9618
Alucinação:          0.67

🔍 O modelo é bom!


#MODELO 1

Alterações realizadas no Modelo Inicial:
*   Alteração no prompt

##Criação da cadeia RAG com LangChain usando PromptTemplate (customizado)

###PromptTemplate

In [657]:
qa_prompt_1 = PromptTemplate.from_template(
"""
Você é um assistente especialista em dados de planilhas. Com base no {context} do Excel, responda com precisão à pergunta: {question}

Responder sempre em português do Brasil;
Responder com até 20 palavras;
NÃO inventar respostas.
Seja direto e técnico.
Se a resposta não estiver clara no contexto, dizer: "Informação não encontrada no contexto."
"""
)

###Cadeia de documentos com Runnable Passthrough

In [658]:
# Juntar documentos em um só contexto
document_chain_1 = RunnablePassthrough() | qa_prompt_1 | llm

###Recuperador

In [659]:
# Buscar documentos com score e filtrar por limiar
def retrieve_with_threshold_fn(pergunta, threshold=0.7):
    # Recupera documentos com seus scores
    resposta_obtida_1 = vectorstore.similarity_search_with_score(pergunta, k=10)
    # Filtra por similaridade mínima
    filtered_docs_1 = [doc for doc, score in resposta_obtida_1 if score >= threshold]
    # Se nada passou do limiar, retorna os top-3 independente do score
    if not filtered_docs_1:
       # print("⚠️ Nenhum documento acima do limiar. Usando os 3 mais similares disponíveis.")
        filtered_docs_1 = [doc for doc, _ in resposta_obtida_1[:3]]

    return filtered_docs_1[:5]  # Retorna os 5 melhores após filtro

###Fluxo de RAG usando RunnablePassthrought

In [1009]:
# Função inteligente para juntar contexto (para debug, se quiser)
def juntar_contexto(docs):
    if not docs:
        raise ValueError("⚠️ Nenhum contexto relevante foi encontrado para essa pergunta.")

    first = docs[0]
    if isinstance(first, str):
        return "\n\n".join(docs)
    elif hasattr(first, "page_content"):
        return "\n\n".join(doc.page_content for doc in docs)
    else:
        raise ValueError(f"❌ Formato desconhecido: {type(first)}")

In [1010]:
# Criação da retrieval_chain final: Faz a pergunta ➔ busca documentos ➔ junta contexto ➔ consulta o modelo.
retrieval_chain_1 = RunnablePassthrough().assign(
    question=lambda x: x["question"]
).assign(
    documents=lambda inputs: retrieve_with_threshold_fn(inputs["question"], threshold=0.7)
).assign(
    context=lambda inputs: juntar_contexto(inputs["documents"])
).assign(
    answer=lambda inputs: document_chain_1.invoke({
        "context": inputs["context"],
        "question": inputs["question"]
    })
)

###Chatbot interativo

In [662]:
#Função RAG que traz as respostas obtidas pelo modelo:
def rag_with_context(pergunta: str):
    max_retries = 3
    attempt = 0

    while attempt < max_retries:
        try:
            resposta_obtida_1 = retrieval_chain_1.invoke({"question": pergunta}) #LangChain que recupera e gera texto.

            # Tratamento das respostas obtidas e conteúdo
            if isinstance(resposta_obtida_1, dict):
                conteudo_1 = resposta_obtida_1.get("answer", resposta_obtida_1.get("output", resposta_obtida_1))
            else:
                conteudo_1 = resposta_obtida_1

            if hasattr(conteudo_1, "content"):
                resposta_texto_1 = conteudo_1.content
            elif isinstance(conteudo_1, str):
                resposta_texto_1 = conteudo_1
            else:
                resposta_texto_1 = str(conteudo_1)

            # Contexto
            contexto_usado_1 = resposta_obtida_1.get("context", "⚠️ Contexto não encontrado")
            return resposta_texto_1, contexto_usado_1

#resposta_obtida = resposta original, que pode vir em vários formatos.
#resposta_texto = texto limpo e padronizado, em formato strm pronto para ser retornado.

        except RateLimitError:
            attempt += 1
            if attempt < max_retries:
                print(f"🚫 Limite de requisições atingido. Tentando novamente em 10 segundos... (tentativa {attempt}/{max_retries})")
                time.sleep(10)
            else:
                print("❌ Limite de tentativas atingido. Tente novamente mais tarde.")
                return "", ""
        except Exception as e:
            print(f"⚠️ Erro inesperado: {e}")
            return "", ""

In [663]:
#Loop de perguntas

print("\n✅ Planilha carregada! Faça suas perguntas sobre os dados.")

while True:      #Para inserir perguntas manualmente
    query = input("\n❓ Digite sua pergunta ou 'sair': ")
    if query.lower() in ["sair", "exit", "quit"]:
        print("👋 Encerrando...")
        break

    resposta_texto_1, contexto_usado_1 = rag_with_context(query)

    if resposta_texto_1:
        print(f"\n 🤖 Resposta do bot:\n{resposta_texto_1}")


✅ Planilha carregada! Faça suas perguntas sobre os dados.

❓ Digite sua pergunta ou 'sair': Dê um exemplo de lipídio ionizável

 🤖 Resposta do bot:
Um exemplo de lipídio ionizável é o SM-102.

❓ Digite sua pergunta ou 'sair': Em qual vacina o SM-102 é usado

 🤖 Resposta do bot:
O SM-102 é usado na vacina da Moderna contra a COVID-19.

❓ Digite sua pergunta ou 'sair': Qual vacina contém N1-metilpseudourina

 🤖 Resposta do bot:
A vacina que contém N1-metilpseudourina é a tozinameran (Pfizer-BioNTech) e a elasomeran (Moderna).

❓ Digite sua pergunta ou 'sair': Qual a função da LNP

 🤖 Resposta do bot:
A função da LNP é atuar como veículo de entrega para o RNA, protegendo-o da degradação e facilitando sua entrada nas células.

❓ Digite sua pergunta ou 'sair': Como é formada uma LNP

 🤖 Resposta do bot:
Uma LNP é formada por lipídios ionizáveis, fosfolipídios, colesterol e é revestida por lipídios peguilados.

❓ Digite sua pergunta ou 'sair': Dê exemplo de vacina com saRNA

 🤖 Resposta do 

##Métricas da qualidade das respostas e do modelo

#### a) Aplicação das métricas

In [1011]:
#Normalização do texto:
def normalize(resposta_texto):
    resposta_texto = resposta_texto.lower()  # Deixa tudo minúsculo
    resposta_texto = re.sub(r'[^\w\s]', '', resposta_texto)  # Remove pontuação
    resposta_texto = re.sub(r'\s+', ' ', resposta_texto) # Substitui múltiplos espaços por um único espaço
    return resposta_texto.strip()  # Remove espaços nas bordas

In [1012]:
def extract_entities(resposta_texto):
    doc = nlp(resposta_texto)
    return {ent.text.lower() for ent in doc.ents}

In [1013]:
#BLEU (Bilingual Evaluation Understudy)
#Mede precisão = quantos n-gramas da resposta gerada aparecem na referência.
def calc_bleu(resposta_texto, resposta_esperada):
    smooth = SmoothingFunction().method4
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        ref_tokens = [ref.strip().split()]  # BLEU espera lista de listas
        pred_tokens = pred.strip().split()
        score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smooth)
        scores.append(score)

    return np.mean(scores)

In [1014]:
# ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
#Mede recall = quanto da referência aparece na resposta
def calc_rouge(resposta_texto, resposta_esperada):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        score = scorer.score(ref.strip(), pred.strip())
        scores.append(score['rougeL'].fmeasure)

    return np.mean(scores)

In [1015]:
# Coverage Score (quanto da resposta esperada estava na resposta gerada)
def calc_coverage(resposta_texto, resposta_esperada):
    ref_tokens = normalize(resposta_esperada).split()
    pred_tokens = normalize(resposta_texto).split()
    common = set(ref_tokens) & set(pred_tokens)
    if not common:
        return 0.0
    return len(common) / len(ref_tokens)

In [1016]:
# Partial Match (modelo aceito se cobrir pelo menos 50% da resposta)
def calc_partial_match(resposta_texto, resposta_esperada, threshold=0.5):
    coverage = calc_coverage(resposta_texto, resposta_esperada)
    return int(coverage >= threshold)

In [1017]:
#Taxa de alucinação = Percentual de respostas inventadas ou que fugiram do contexto
def check_hallucination(resposta_texto, contexto_usado):
    if "nenhuma informação suficiente" in resposta_texto.lower():
        return False  # isso NÃO é alucinação, é fallback correto

    resposta_entidades = extract_entities(resposta_texto)
    contexto_entidades = extract_entities(contexto_usado)

    # Verifica se todas as entidades na resposta estão presentes no contexto
    return not resposta_entidades.issubset(contexto_entidades)

In [1018]:
def calc_bertscore(resposta_texto, resposta_esperada):
    P, R, F1 = score([resposta_texto], [resposta_esperada], lang='pt', verbose=False)
    return F1.item()

####b) Avaliação das respostas obtidas

In [1019]:
# Conjunto de perguntas e respostas esperadas
avaliacao = [
     {"pergunta": "Dê um exemplo de lipídio ionizável", "resposta_esperada": "Um exemplo de lipídio ionizável é o SM-102"},
     {"pergunta": "Em qual vacina o SM-102 é usado", "resposta_esperada":"O SM-102 é usado na vacina da Moderna COVID-19"},
     {"pergunta": "Qual vacina contém N1-metilpseudourina", "resposta_esperada": "A vacina que contém N1-metilpseudouridina é a tozinameran (Pfizer-BioNTech) e elasomeran (Moderna)."},
     {"pergunta": "Qual a função da LNP?", "resposta_esperada": "A função da LNP é atuar como veículo de entrega para o RNA, protegendo-o da degradação e facilitando sua entrada nas células."},
     {"pergunta": "Como é formada uma LNP?", "resposta_esperada": "A LNP é formada por lipídios ionizáveis, fosfolipídios, colesterol e é revestida por lipídios peguilados"},
     {"pergunta": "Dê exemplo de vacina com saRNA", "resposta_esperada": "Um exemplo de vacina com saRNA é Gemcovac, autorizada na Índia em 2022"}
]

In [1020]:
print("\n🔍 Avaliação das Respostas:")
print("=" * 80)

#Inicialização das métricas globais
total_bleu_1, total_rouge_1, total_coverage_1, total_partial_1, total_bertscore_1 = 0, 0, 0, 0, 0
hallucinated_1 = 0

#Avaliação automática de respostas = compara as respostas obtidas x esperadas
for exemplo_1 in avaliacao:
    pergunta = exemplo_1["pergunta"]
    resposta_esperada_1 = exemplo_1["resposta_esperada"]
    try:
        resposta_texto_1, contexto_usado_1 = rag_with_context(pergunta)
    except Exception as e:
        print(f"❌ Erro ao buscar resposta para '{pergunta}': {e}")
        resposta_texto_1, contexto_usado_1 = "", ""

#Calcula as métricas da resposta
    bleu = calc_bleu(resposta_texto_1, resposta_esperada_1)
    rouge = calc_rouge(resposta_texto_1, resposta_esperada_1)
    coverage = calc_coverage(resposta_texto_1, resposta_esperada_1)
    partial = calc_partial_match(resposta_texto_1, resposta_esperada_1, threshold=0.5)
    aluc = check_hallucination(resposta_texto_1, contexto_usado_1)
    bertscore = calc_bertscore(resposta_texto_1, resposta_esperada_1)

#Acumula os resultados nas variáveis globais
    total_bleu_1 += bleu
    total_rouge_1 += rouge
    total_coverage_1 += coverage
    total_partial_1 += partial
    total_bertscore_1 += bertscore
    hallucinated_1 += int(aluc)

# Exibe avaliação por pergunta
    print(f"Pergunta: {pergunta}")
    print(f"Resposta esperada: {resposta_esperada_1}")
    print(f"Resposta do modelo: {resposta_texto_1}")
    print(f"Contexto: {contexto_usado_1[:200]}...")  # Mostra só o início
    print(f"bleu={bleu:.4f} | rouge={rouge:.4f} | Coverage={coverage:.2f} | Partial={partial} | BERTScore={bertscore:.4f} | Alucinação={'Sim' if aluc else 'Não'}")
    print("-" * 80)


🔍 Avaliação das Respostas:
Pergunta: Dê um exemplo de lipídio ionizável
Resposta esperada: Um exemplo de lipídio ionizável é o SM-102
Resposta do modelo: Um exemplo de lipídio ionizável é o SM-102.
Contexto: [Planilha2] Ionisable lipids like SM-102 hold a relatively (/ close to) neutral charge at physiological pH but are positively charged within the nanoparticle (the amine group is protonated to form an ...
bleu=0.8333 | rouge=0.7381 | Coverage=1.00 | Partial=1 | BERTScore=0.9823 | Alucinação=Não
--------------------------------------------------------------------------------
Pergunta: Em qual vacina o SM-102 é usado
Resposta esperada: O SM-102 é usado na vacina da Moderna COVID-19
Resposta do modelo: O SM-102 é usado na vacina da Moderna contra a COVID-19.
Contexto: [Planilha2] SM-102 is a synthetic amino lipid which is used in combination with other lipids to form lipid nanoparticles.[1] These are used for the delivery of mRNA-based vaccines,[2][3][4] and in par...
bleu=0.6522 | ro

#### b) Avaliação do modelo

#####***CRITÉRIOS***
#####**Bleu:** O modelo é considerado bom se o Bleu for superior a 0.3.
#####**Rouge:** O modelo é considerado bom se o Rouge for superior a 0.3.
#####**Coverage:** O modelo é considerado bom se o Coverage médio for superior a 0.7.
#####**Partial Match:** O modelo é considerado bom se a taxa de Partial Match for superior a 0.5.
#####**BERTScore:** O modelo é considerado bom se o BERTScore for superior a 0.8.
#####**Alucinação:** O modelo é considerado bom se a taxa de alucinação for menor que 0.7.

#####***AVALIAÇÃO***

In [1021]:
n = len(avaliacao)

# Função e critérios para avaliar se o modelo é bom, com base nas métricas finais
def avaliar_modelo_1(total_bleu_1, total_rouge_1, total_coverage_1, total_partial_1, total_bertscore_1, hallucinated_1, n):
    bleu_threshold = 0.3
    rouge_threshold = 0.3
    coverage_threshold = 0.7
    partial_threshold = 0.5
    bertscore_threshold = 0.8
    hallucination_threshold = 0.7

    # Calculando as métricas médias
    avg_bleu_1 = total_bleu_1 / n
    avg_rouge_1 = total_rouge_1/ n
    avg_coverage_1 = total_coverage_1 / n
    avg_partial_1 = total_partial_1 / n
    avg_bertscore_1 = total_bertscore_1 / n
    avg_hallucinated_1 = hallucinated_1 / n

    # Exibindo as métricas finais
    print("\nMÉTRICAS FINAIS - Modelo 1:")
    print(f"Bleu:                {avg_bleu_1:.4f}")
    print(f"Rouge:               {avg_rouge_1:.4f}")
    print(f"Coverage médio:      {avg_coverage_1:.2f}")
    print(f"Partial Match taxa:  {avg_partial_1:.2f}")
    print(f"BERTScore médio:     {avg_bertscore_1:.4f}")
    print(f"Alucinação:          {avg_hallucinated_1:.2f}")

    # Avaliando o modelo com base nas métricas
    if avg_bleu_1 >= bleu_threshold and avg_rouge_1 >= rouge_threshold and avg_coverage_1 >= coverage_threshold and \
       avg_partial_1 >= partial_threshold and avg_bertscore_1 >= bertscore_threshold and avg_hallucinated_1 <= hallucination_threshold:
        print("\n🔍 O modelo é bom!")
    else:
        print("\n⚠️ O modelo precisa de melhorias.")

#Chamada da função para avaliar o modelo
avaliar_modelo_1(total_bleu_1, total_rouge_1, total_coverage_1, total_partial_1, total_bertscore_1, hallucinated_1, n)


MÉTRICAS FINAIS - Modelo 1:
Bleu:                0.5111
Rouge:               0.4742
Coverage médio:      0.91
Partial Match taxa:  1.00
BERTScore médio:     0.9696
Alucinação:          0.67

🔍 O modelo é bom!


#MODELO 2

Alterações realizadas no Modelo Inicial:
*   Alteração do Threshold de 0.7 para 0.9 do recuperador *def retrieve_with_threshold_fn(pergunta, threshold)*

##Criação da cadeia RAG com LangChain usando PromptTemplate (customizado)

###PromptTemplate

In [841]:
qa_prompt_2 = PromptTemplate.from_template(
"""
Você é um assistente especialista em dados de planilhas. Com base no {context} do Excel, responda com precisão à pergunta: {question}
Responda de forma clara, objetiva e com linguagem técnica acessível.
"""
)

###Cadeia de documentos com Runnable Passthrough

In [842]:
# Juntar documentos em um só contexto
document_chain_2 = RunnablePassthrough() | qa_prompt_2 | llm

###Recuperador

In [843]:
# Buscar documentos com score e filtrar por limiar
def retrieve_with_threshold_fn(pergunta, threshold=0.9):
    # Recupera documentos com seus scores
    resposta_obtida_2 = vectorstore.similarity_search_with_score(pergunta, k=10)
    # Filtra por similaridade mínima
    filtered_docs_2 = [doc for doc, score in resposta_obtida_2 if score >= threshold]
    # Se nada passou do limiar, retorna os top-3 independente do score
    if not filtered_docs_2:
       # print("⚠️ Nenhum documento acima do limiar. Usando os 3 mais similares disponíveis.")
        filtered_docs_2 = [doc for doc, _ in resposta_obtida_2[:3]]

    return filtered_docs_2[:5]  # Retorna os 5 melhores após filtro

###Fluxo de RAG usando RunnablePassthrought

In [844]:
# Função inteligente para juntar contexto (para debug, se quiser)
def juntar_contexto(docs):
    if not docs:
        raise ValueError("⚠️ Nenhum contexto relevante foi encontrado para essa pergunta.")

    first = docs[0]
    if isinstance(first, str):
        return "\n\n".join(docs)
    elif hasattr(first, "page_content"):
        return "\n\n".join(doc.page_content for doc in docs)
    else:
        raise ValueError(f"❌ Formato desconhecido: {type(first)}")

In [847]:
# Criação da retrieval_chain final: Faz a pergunta ➔ busca documentos ➔ junta contexto ➔ consulta o modelo.
retrieval_chain_2 = RunnablePassthrough().assign(
    question=lambda x: x["question"]
).assign(
    documents=lambda inputs: retrieve_with_threshold_fn(inputs["question"], threshold=0.9)
).assign(
    context=lambda inputs: juntar_contexto(inputs["documents"])
).assign(
    answer=lambda inputs: document_chain_2.invoke({
        "context": inputs["context"],
        "question": inputs["question"]
    })
)

###Chatbot interativo

In [848]:
#Função RAG que traz as respostas obtidas pelo modelo:
def rag_with_context(pergunta: str):
    max_retries = 3
    attempt = 0

    while attempt < max_retries:
        try:
            resposta_obtida_2 = retrieval_chain_2.invoke({"question": pergunta}) #LangChain que recupera e gera texto.

            # Tratamento das respostas obtidas e conteúdo
            if isinstance(resposta_obtida_2, dict):
                conteudo = resposta_obtida_2.get("answer", resposta_obtida_2.get("output", resposta_obtida_2))
            else:
                conteudo = resposta_obtida_2

            if hasattr(conteudo, "content"):
                resposta_texto_2 = conteudo.content
            elif isinstance(conteudo, str):
                resposta_texto_2 = conteudo
            else:
                resposta_texto_2 = str(conteudo)

            # Contexto
            contexto_usado_2 = resposta_obtida_2.get("context", "⚠️ Contexto não encontrado")
            return resposta_texto_2, contexto_usado_2

#resposta_obtida = resposta original, que pode vir em vários formatos.
#resposta_texto = texto limpo e padronizado, em formato strm pronto para ser retornado.

        except RateLimitError:
            attempt += 1
            if attempt < max_retries:
                print(f"🚫 Limite de requisições atingido. Tentando novamente em 10 segundos... (tentativa {attempt}/{max_retries})")
                time.sleep(10)
            else:
                print("❌ Limite de tentativas atingido. Tente novamente mais tarde.")
                return "", ""
        except Exception as e:
            print(f"⚠️ Erro inesperado: {e}")
            return "", ""

In [849]:
#Loop de perguntas

print("\n✅ Planilha carregada! Faça suas perguntas sobre os dados.")

while True:      #Para inserir perguntas manualmente
    query = input("\n❓ Digite sua pergunta ou 'sair': ")
    if query.lower() in ["sair", "exit", "quit"]:
        print("👋 Encerrando...")
        break

    resposta_texto_2, contexto_usado_2 = rag_with_context(query)

    if resposta_texto_2:
        print(f"\n 🤖 Resposta do bot:\n{resposta_texto_2}")


✅ Planilha carregada! Faça suas perguntas sobre os dados.

❓ Digite sua pergunta ou 'sair': Dê um exemplo de lipídio ionizável

 🤖 Resposta do bot:
Um exemplo de lipídio ionizável é o SM-102, que possui uma carga relativamente neutra em pH fisiológico, mas se torna positivamente carregado dentro da nanopartícula devido à protonação do grupo amina para formar um cátion de amônio. Isso permite que ele se ligue à espinha dorsal negativamente carregada do mRNA.

❓ Digite sua pergunta ou 'sair': Em qual vacina o SM-102 é usado

 🤖 Resposta do bot:
O SM-102 é usado na vacina da Moderna contra a COVID-19. Ele é um amino lipídio sintético que faz parte do sistema de entrega de medicamentos da vacina.

❓ Digite sua pergunta ou 'sair': Qual vacina contém N1-metilpseudourina

 🤖 Resposta do bot:
A vacina que contém N1-metilpseudourina é a tozinameran (Pfizer-BioNTech) e a elasomeran (Moderna), que são vacinas de mRNA contra a COVID-19. A N1-metilpseudourina é utilizada nessas vacinas para propor

##Métricas da qualidade das respostas e do modelo

#### a) Aplicação das métricas

In [946]:
#Normalização do texto:
def normalize(resposta_texto):
    resposta_texto = resposta_texto.lower()  # Deixa tudo minúsculo
    resposta_texto = re.sub(r'[^\w\s]', '', resposta_texto)  # Remove pontuação
    resposta_texto = re.sub(r'\s+', ' ', resposta_texto) # Substitui múltiplos espaços por um único espaço
    return resposta_texto.strip()  # Remove espaços nas bordas

In [947]:
def extract_entities(resposta_texto):
    doc = nlp(resposta_texto)
    return {ent.text.lower() for ent in doc.ents}

In [948]:
#BLEU (Bilingual Evaluation Understudy)
#Mede precisão = quantos n-gramas da resposta gerada aparecem na referência.
def calc_bleu(resposta_texto, resposta_esperada):
    smooth = SmoothingFunction().method4
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        ref_tokens = [ref.strip().split()]  # BLEU espera lista de listas
        pred_tokens = pred.strip().split()
        score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smooth)
        scores.append(score)

    return np.mean(scores)

In [949]:
# ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
#Mede recall = quanto da referência aparece na resposta
def calc_rouge(resposta_texto, resposta_esperada):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        score = scorer.score(ref.strip(), pred.strip())
        scores.append(score['rougeL'].fmeasure)

    return np.mean(scores)

In [950]:
# Coverage Score (quanto da resposta esperada estava na resposta gerada)
def calc_coverage(resposta_texto, resposta_esperada):
    ref_tokens = normalize(resposta_esperada).split()
    pred_tokens = normalize(resposta_texto).split()
    common = set(ref_tokens) & set(pred_tokens)
    if not common:
        return 0.0
    return len(common) / len(ref_tokens)

In [951]:
# Partial Match (modelo aceito se cobrir pelo menos 50% da resposta)
def calc_partial_match(resposta_texto, resposta_esperada, threshold=0.5):
    coverage = calc_coverage(resposta_texto, resposta_esperada)
    return int(coverage >= threshold)

In [952]:
#Taxa de alucinação = Percentual de respostas inventadas ou que fugiram do contexto
def check_hallucination(resposta_texto, contexto_usado):
    if "nenhuma informação suficiente" in resposta_texto.lower():
        return False  # isso NÃO é alucinação, é fallback correto

    resposta_entidades = extract_entities(resposta_texto)
    contexto_entidades = extract_entities(contexto_usado)

    # Verifica se todas as entidades na resposta estão presentes no contexto
    return not resposta_entidades.issubset(contexto_entidades)

In [953]:
def calc_bertscore(resposta_texto, resposta_esperada):
    P, R, F1 = score([resposta_texto], [resposta_esperada], lang='pt', verbose=False)
    return F1.item()

####b) Avaliação das respostas obtidas

In [954]:
# Conjunto de perguntas e respostas esperadas
avaliacao = [
     {"pergunta": "Dê um exemplo de lipídio ionizável", "resposta_esperada": "Um exemplo de lipídio ionizável é o SM-102"},
     {"pergunta": "Em qual vacina o SM-102 é usado", "resposta_esperada":"O SM-102 é usado na vacina da Moderna COVID-19"},
     {"pergunta": "Qual vacina contém N1-metilpseudourina", "resposta_esperada": "A vacina que contém N1-metilpseudouridina é a tozinameran (Pfizer-BioNTech) e elasomeran (Moderna)."},
     {"pergunta": "Qual a função da LNP?", "resposta_esperada": "A função da LNP é atuar como veículo de entrega para o RNA, protegendo-o da degradação e facilitando sua entrada nas células."},
     {"pergunta": "Como é formada uma LNP?", "resposta_esperada": "A LNP é formada por lipídios ionizáveis, fosfolipídios, colesterol e é revestida por lipídios peguilados"},
     {"pergunta": "Dê exemplo de vacina com saRNA", "resposta_esperada": "Um exemplo de vacina com saRNA é Gemcovac, autorizada na Índia em 2022"}
]

In [955]:
print("\n🔍 Avaliação das Respostas:")
print("=" * 80)

#Inicialização das métricas globais
total_bleu_2, total_rouge_2, total_coverage_2, total_partial_2, total_bertscore_2 = 0, 0, 0, 0, 0
hallucinated_2 = 0

#Avaliação automática de respostas = compara as respostas obtidas x esperadas
for exemplo_2 in avaliacao:
    pergunta = exemplo_2["pergunta"]
    resposta_esperada_2 = exemplo_2["resposta_esperada"]
    try:
        resposta_texto_2, contexto_usado_2 = rag_with_context(pergunta)
    except Exception as e:
        print(f"❌ Erro ao buscar resposta para '{pergunta}': {e}")
        resposta_texto_2, contexto_usado_2 = "", ""

#Calcula as métricas da resposta
    bleu = calc_bleu(resposta_texto_2, resposta_esperada_2)
    rouge = calc_rouge(resposta_texto_2, resposta_esperada_2)
    coverage = calc_coverage(resposta_texto_2, resposta_esperada_2)
    partial = calc_partial_match(resposta_texto_2, resposta_esperada_2, threshold=0.5)
    aluc = check_hallucination(resposta_texto_2, contexto_usado_2)
    bertscore = calc_bertscore(resposta_texto_2, resposta_esperada_2)

#Acumula os resultados nas variáveis globais
    total_bleu_2 += bleu
    total_rouge_2 += rouge
    total_coverage_2 += coverage
    total_partial_2 += partial
    total_bertscore_2 += bertscore
    hallucinated_2 += int(aluc)

# Exibe avaliação por pergunta
    print(f"Pergunta: {pergunta}")
    print(f"Resposta esperada: {resposta_esperada_2}")
    print(f"Resposta do modelo: {resposta_texto_2}")
    print(f"Contexto: {contexto_usado_2[:200]}...")  # Mostra só o início
    print(f"bleu={bleu:.4f} | rouge={rouge:.4f} | Coverage={coverage:.2f} | Partial={partial} | BERTScore={bertscore:.4f} | Alucinação={'Sim' if aluc else 'Não'}")
    print("-" * 80)


🔍 Avaliação das Respostas:
Pergunta: Dê um exemplo de lipídio ionizável
Resposta esperada: Um exemplo de lipídio ionizável é o SM-102
Resposta do modelo: Um exemplo de lipídio ionizável é o SM-102, que possui carga positiva dentro de nanopartículas.
Contexto: [Planilha2] Ionisable lipids like SM-102 hold a relatively (/ close to) neutral charge at physiological pH but are positively charged within the nanoparticle (the amine group is protonated to form an ...
bleu=0.8333 | rouge=0.7381 | Coverage=1.00 | Partial=1 | BERTScore=0.8945 | Alucinação=Não
--------------------------------------------------------------------------------
Pergunta: Em qual vacina o SM-102 é usado
Resposta esperada: O SM-102 é usado na vacina da Moderna COVID-19
Resposta do modelo: O SM-102 é usado na vacina da Moderna contra a COVID-19.
Contexto: [Planilha2] SM-102 is a synthetic amino lipid which is used in combination with other lipids to form lipid nanoparticles.[1] These are used for the delivery of mRNA-bas

#### b) Avaliação do modelo

#####***CRITÉRIOS***
#####**Bleu:** O modelo é considerado bom se o Bleu for superior a 0.3.
#####**Rouge:** O modelo é considerado bom se o Rouge for superior a 0.3.
#####**Coverage:** O modelo é considerado bom se o Coverage médio for superior a 0.7.
#####**Partial Match:** O modelo é considerado bom se a taxa de Partial Match for superior a 0.5.
#####**BERTScore:** O modelo é considerado bom se o BERTScore for superior a 0.8.
#####**Alucinação:** O modelo é considerado bom se a taxa de alucinação for menor que 0.7.

#####***AVALIAÇÃO***

In [956]:
n = len(avaliacao)

# Função e critérios para avaliar se o Modelo 2 é bom, com base nas métricas finais
def avaliar_modelo_2(total_bleu_2, total_rouge_2, total_coverage_2, total_partial_2, total_bertscore_2, hallucinated_2, n):
    bleu_threshold = 0.3
    rouge_threshold = 0.3
    coverage_threshold = 0.7
    partial_threshold = 0.5
    bertscore_threshold = 0.8
    hallucination_threshold = 0.7

    # Calculando as métricas médias para o Modelo 2
    avg_bleu_2 = total_bleu_2 / n
    avg_rouge_2 = total_rouge_2 / n
    avg_coverage_2 = total_coverage_2 / n
    avg_partial_2 = total_partial_2 / n
    avg_bertscore_2 = total_bertscore_2 / n
    avg_hallucinated_2 = hallucinated_2 / n

    # Exibindo as métricas finais do Modelo 2
    print("\nMÉTRICAS FINAIS - Modelo 2:")
    print(f"Bleu:                 {avg_bleu_2:.4f}")
    print(f"Rouge:                {avg_rouge_2:.4f}")
    print(f"Coverage médio:       {avg_coverage_2:.2f}")
    print(f"Partial Match taxa:   {avg_partial_2:.2f}")
    print(f"BERTScore médio:      {avg_bertscore_2:.4f}")
    print(f"Alucinação:           {avg_hallucinated_2:.2f}")

    # Avaliando o Modelo 2 com base nas métricas
    if avg_bleu_2 >= bleu_threshold and avg_rouge_2 >= rouge_threshold and avg_coverage_2 >= coverage_threshold and \
       avg_partial_2 >= partial_threshold and avg_bertscore_2 >= bertscore_threshold and avg_hallucinated_2 <= hallucination_threshold:
        print("\n🔍 O Modelo 2 é bom!")
    else:
        print("\n⚠️ O Modelo 2 precisa de melhorias.")

# Chamada da função para avaliar o Modelo 2
avaliar_modelo_2(total_bleu_2, total_rouge_2, total_coverage_2, total_partial_2, total_bertscore_2, hallucinated_2, n)


MÉTRICAS FINAIS - Modelo 2:
Bleu:                 0.5435
Rouge:                0.5035
Coverage médio:       0.87
Partial Match taxa:   1.00
BERTScore médio:      0.9472
Alucinação:           0.67

🔍 O Modelo 2 é bom!


#MODELO 3

Alterações realizadas no modelo inicial:
*   Alteração no prompt
*   Alteração no Threshold de 0.7 para 0.9

##Criação da cadeia RAG com LangChain usando PromptTemplate (customizado)

###PromptTemplate

In [865]:
qa_prompt_3 = PromptTemplate.from_template(
"""
Você é um assistente especialista em dados de planilhas. Com base no {context} do Excel, responda com precisão à pergunta: {question}

Responder sempre em português do Brasil;
Responder com até 20 palavras;
NÃO inventar respostas.
Seja direto e técnico.
Se a resposta não estiver clara no contexto, dizer: "Informação não encontrada no contexto."
"""
)

### Cadeia de documentos com Runnable Passthrough

In [863]:
# Juntar documentos em um só contexto
document_chain_3 = RunnablePassthrough() | qa_prompt_3 | llm

###Recuperador

In [864]:
# Buscar documentos com score e filtrar por limiar
def retrieve_with_threshold_fn(pergunta, threshold=0.9):
    # Recupera documentos com seus scores
    resposta_obtida_3 = vectorstore.similarity_search_with_score(pergunta, k=10)
    # Filtra por similaridade mínima
    filtered_docs_3 = [doc for doc, score in resposta_obtida_3 if score >= threshold]
    # Se nada passou do limiar, retorna os top independente do score
    if not filtered_docs_3:
       # print("⚠️ Nenhum documento acima do limiar. Usando os 3 mais similares disponíveis.")
        filtered_docs_3 = [doc for doc, _ in resposta_obtida_3[:3]]

    return filtered_docs_3[:5]  # Retorna os 5 melhores após filtro

###Fluxo de RAG usando RunnablePassthrought

In [698]:
# Função inteligente para juntar contexto (para debug, se quiser)
def juntar_contexto(docs):
    if not docs:
        raise ValueError("⚠️ Nenhum contexto relevante foi encontrado para essa pergunta.")

    first = docs[0]
    if isinstance(first, str):
        return "\n\n".join(docs)
    elif hasattr(first, "page_content"):
        return "\n\n".join(doc.page_content for doc in docs)
    else:
        raise ValueError(f"❌ Formato desconhecido: {type(first)}")

In [866]:
# Criação da retrieval_chain final: Faz a pergunta ➔ busca documentos ➔ junta contexto ➔ consulta o modelo.
retrieval_chain_3 = RunnablePassthrough().assign(
    question=lambda x: x["question"]
).assign(
    documents=lambda inputs: retrieve_with_threshold_fn(inputs["question"], threshold=0.9)
).assign(
    context=lambda inputs: juntar_contexto(inputs["documents"])
).assign(
    answer=lambda inputs: document_chain_3.invoke({
        "context": inputs["context"],
        "question": inputs["question"]
    })
)

###Chatbot interativo

In [867]:
#Função RAG que traz as respostas obtidas pelo modelo:
def rag_with_context(pergunta: str):
    max_retries = 3
    attempt = 0

    while attempt < max_retries:
        try:
            resposta_obtida_3 = retrieval_chain_3.invoke({"question": pergunta}) #LangChain que recupera e gera texto.

            # Tratamento das respostas obtidas e conteúdo
            if isinstance(resposta_obtida_3, dict):
                conteudo_3 = resposta_obtida_3.get("answer", resposta_obtida_3.get("output", resposta_obtida_3))
            else:
                conteudo_3 = resposta_obtida_3

            if hasattr(conteudo_3, "content"):
                resposta_texto_3 = conteudo_3.content.strip()
            elif isinstance(conteudo_3, str):
                resposta_texto_3 = conteudo_3.strip()
            else:
                resposta_texto_3 = str(conteudo_3).strip()

            # Contexto
            contexto_usado_3 = resposta_obtida_3.get("context", "⚠️ Contexto não encontrado")
            return resposta_texto_3, contexto_usado_3

#resposta_obtida = resposta original, que pode vir em vários formatos.
#resposta_texto = texto limpo e padronizado, em formato strm pronto para ser retornado.

        except RateLimitError:
            attempt += 1
            if attempt < max_retries:
                print(f"🚫 Limite de requisições atingido. Tentando novamente em 10 segundos... (tentativa {attempt}/{max_retries})")
                time.sleep(10)
            else:
                print("❌ Limite de tentativas atingido. Tente novamente mais tarde.")
                return "", ""
        except Exception as e:
            print(f"⚠️ Erro inesperado: {e}")
            return "", ""

In [868]:
#Loop de perguntas

print("\n✅ Planilha carregada! Faça suas perguntas sobre os dados.")

while True:      #Para inserir perguntas manualmente
    query = input("\n❓ Digite sua pergunta ou 'sair': ")
    if query.lower() in ["sair", "exit", "quit"]:
        print("👋 Encerrando...")
        break

    resposta_texto_3, contexto_usado_3 = rag_with_context(query)

    if resposta_texto_3:
        print(f"\n 🤖 Resposta do bot:\n{resposta_texto_3}")


✅ Planilha carregada! Faça suas perguntas sobre os dados.

❓ Digite sua pergunta ou 'sair': Dê um exemplo de lipídio ionizável

 🤖 Resposta do bot:
Um exemplo de lipídio ionizável é o SM-102.

❓ Digite sua pergunta ou 'sair': Em qual vacina o SM-102 é usado

 🤖 Resposta do bot:
O SM-102 é usado na vacina da Moderna contra a COVID-19.

❓ Digite sua pergunta ou 'sair': Qual vacina contém N1-metilpseudourina

 🤖 Resposta do bot:
A vacina que contém N1-metilpseudouridina é a tozinameran (Pfizer-BioNTech).

❓ Digite sua pergunta ou 'sair': Qual a função da LNP

 🤖 Resposta do bot:
A função da LNP é atuar como veículo de entrega para o RNA, protegendo-o da degradação e facilitando sua entrada nas células.

❓ Digite sua pergunta ou 'sair': Como é formada uma LNP

 🤖 Resposta do bot:
Uma LNP é formada por lipídios ionizáveis, fosfolipídios, colesterol e é revestida por lipídios peguilados.

❓ Digite sua pergunta ou 'sair': Dê exemplo de vacina com saRNA

 🤖 Resposta do bot:
Exemplo de vacina 

##Métricas da qualidade das respostas e do modelo

#### a) Aplicação das métricas

In [1037]:
#Normalização do texto:
def normalize(resposta_texto):
    resposta_texto = resposta_texto.lower()  # Deixa tudo minúsculo
    resposta_texto = re.sub(r'[^\w\s]', '', resposta_texto)  # Remove pontuação
    resposta_texto = re.sub(r'\s+', ' ', resposta_texto) # Substitui múltiplos espaços por um único espaço
    return resposta_texto.strip()  # Remove espaços nas bordas

In [1038]:
def extract_entities(resposta_texto):
    doc = nlp(resposta_texto)
    return {ent.text.lower() for ent in doc.ents}

In [1039]:
#BLEU (Bilingual Evaluation Understudy)
#Mede precisão = quantos n-gramas da resposta gerada aparecem na referência.
def calc_bleu(resposta_texto, resposta_esperada):
    smooth = SmoothingFunction().method4
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        ref_tokens = [ref.strip().split()]  # BLEU espera lista de listas
        pred_tokens = pred.strip().split()
        score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smooth)
        scores.append(score)

    return np.mean(scores)

In [1040]:
# ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
#Mede recall = quanto da referência aparece na resposta
def calc_rouge(resposta_texto, resposta_esperada):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        score = scorer.score(ref.strip(), pred.strip())
        scores.append(score['rougeL'].fmeasure)

    return np.mean(scores)

In [1041]:
# Coverage Score (quanto da resposta esperada estava na resposta gerada)
def calc_coverage(resposta_texto, resposta_esperada):
    ref_tokens = normalize(resposta_esperada).split()
    pred_tokens = normalize(resposta_texto).split()
    common = set(ref_tokens) & set(pred_tokens)
    if not common:
        return 0.0
    return len(common) / len(ref_tokens)

In [1042]:
# Partial Match (modelo aceito se cobrir pelo menos 50% da resposta)
def calc_partial_match(resposta_texto, resposta_esperada, threshold=0.5):
    coverage = calc_coverage(resposta_texto, resposta_esperada)
    return int(coverage >= threshold)

In [1043]:
#Taxa de alucinação = Percentual de respostas inventadas ou que fugiram do contexto
def check_hallucination(resposta_texto, contexto_usado):
    if "nenhuma informação suficiente" in resposta_texto_3.lower():
        return False  # isso NÃO é alucinação, é fallback correto

    resposta_entidades = extract_entities(resposta_texto)
    contexto_entidades = extract_entities(contexto_usado)

    # Verifica se todas as entidades na resposta estão presentes no contexto
    return not resposta_entidades.issubset(contexto_entidades)

In [1044]:
def calc_bertscore(resposta_texto, resposta_esperada):
    P, R, F1 = score([resposta_texto], [resposta_esperada], lang='pt', verbose=False)
    return F1.item()

####b) Avaliação das respostas obtidas

In [1045]:
# Conjunto de perguntas e respostas esperadas
avaliacao = [
     {"pergunta": "Dê um exemplo de lipídio ionizável", "resposta_esperada": "Um exemplo de lipídio ionizável é o SM-102"},
     {"pergunta": "Em qual vacina o SM-102 é usado", "resposta_esperada":"O SM-102 é usado na vacina da Moderna COVID-19"},
     {"pergunta": "Qual vacina contém N1-metilpseudourina", "resposta_esperada": "A vacina que contém N1-metilpseudouridina é a tozinameran (Pfizer-BioNTech) e elasomeran (Moderna)."},
     {"pergunta": "Qual a função da LNP?", "resposta_esperada": "A função da LNP é atuar como veículo de entrega para o RNA, protegendo-o da degradação e facilitando sua entrada nas células."},
     {"pergunta": "Como é formada uma LNP?", "resposta_esperada": "A LNP é formada por lipídios ionizáveis, fosfolipídios, colesterol e é revestida por lipídios peguilados"},
     {"pergunta": "Dê exemplo de vacina com saRNA", "resposta_esperada": "Um exemplo de vacina com saRNA é Gemcovac, autorizada na Índia em 2022"}
]

In [1046]:
print("\n🔍 Avaliação das Respostas:")
print("=" * 80)

#Inicialização das métricas globais
total_bleu_3, total_rouge_3, total_coverage_3, total_partial_3, total_bertscore_3 = 0, 0, 0, 0, 0
hallucinated_3 = 0

#Avaliação automática de respostas = compara as respostas obtidas x esperadas
for exemplo_3 in avaliacao:
    pergunta = exemplo_3["pergunta"]
    resposta_esperada_3 = exemplo_3["resposta_esperada"]
    try:
        resposta_texto_3, contexto_usado_3 = rag_with_context(pergunta)
    except Exception as e:
        print(f"❌ Erro ao buscar resposta para '{pergunta}': {e}")
        resposta_texto_3, contexto_usado_3 = "", ""

#Calcula as métricas da resposta
    bleu = calc_bleu(resposta_texto_3, resposta_esperada_3)
    rouge = calc_rouge(resposta_texto_3, resposta_esperada_3)
    coverage = calc_coverage(resposta_texto_3, resposta_esperada_3)
    partial = calc_partial_match(resposta_texto_3, resposta_esperada_3, threshold=0.5)
    aluc = check_hallucination(resposta_texto_3, contexto_usado_3)
    bertscore = calc_bertscore(resposta_texto_3, resposta_esperada_3)

#Acumula os resultados nas variáveis globais
    total_bleu_3 += bleu
    total_rouge_3 += rouge
    total_coverage_3 += coverage
    total_partial_3 += partial
    total_bertscore_3 += bertscore
    hallucinated_3 += int(aluc)

# Exibe avaliação por pergunta
    print(f"Pergunta: {pergunta}")
    print(f"Resposta esperada: {resposta_esperada_3}")
    print(f"Resposta do modelo: {resposta_texto_3}")
    print(f"Contexto: {contexto_usado_3[:200]}...")  # Mostra só o início
    print(f"bleu={bleu:.4f} | rouge={rouge:.4f} | Coverage={coverage:.2f} | Partial={partial} | BERTScore={bertscore:.4f} | Alucinação={'Sim' if aluc else 'Não'}")
    print("-" * 80)


🔍 Avaliação das Respostas:
Pergunta: Dê um exemplo de lipídio ionizável
Resposta esperada: Um exemplo de lipídio ionizável é o SM-102
Resposta do modelo: Um exemplo de lipídio ionizável é o SM-102.
Contexto: [Planilha2] Ionisable lipids like SM-102 hold a relatively (/ close to) neutral charge at physiological pH but are positively charged within the nanoparticle (the amine group is protonated to form an ...
bleu=0.8333 | rouge=0.7381 | Coverage=1.00 | Partial=1 | BERTScore=0.9823 | Alucinação=Não
--------------------------------------------------------------------------------
Pergunta: Em qual vacina o SM-102 é usado
Resposta esperada: O SM-102 é usado na vacina da Moderna COVID-19
Resposta do modelo: O SM-102 é usado na vacina da Moderna contra a COVID-19.
Contexto: [Planilha2] SM-102 is a synthetic amino lipid which is used in combination with other lipids to form lipid nanoparticles.[1] These are used for the delivery of mRNA-based vaccines,[2][3][4] and in par...
bleu=0.6522 | ro

#### b) Avaliação do modelo

#####***CRITÉRIOS***
#####**Bleu:** O modelo é considerado bom se o Bleu for superior a 0.3.
#####**Rouge:** O modelo é considerado bom se o Rouge for superior a 0.3.
#####**Coverage:** O modelo é considerado bom se o Coverage médio for superior a 0.7.
#####**Partial Match:** O modelo é considerado bom se a taxa de Partial Match for superior a 0.5.
#####**Alucinação:** O modelo é considerado bom se a taxa de alucinação for menor que 0.7.

#####***AVALIAÇÃO***

In [1047]:
n = len(avaliacao)

# Função e critérios para avaliar se o modelo é bom, com base nas métricas finais
def avaliar_modelo_3(total_bleu_3, total_rouge_3, total_coverage_3, total_partial_3, total_bertscore_3, hallucinated_3, n):
    bleu_threshold = 0.3
    rouge_threshold = 0.3
    coverage_threshold = 0.7
    partial_threshold = 0.5
    bertscore_threshold = 0.8
    hallucination_threshold = 0.7

    # Calculando as métricas médias
    avg_bleu_3 = total_bleu_3 / n
    avg_rouge_3 = total_rouge_3 / n
    avg_coverage_3 = total_coverage_3 / n
    avg_partial_3 = total_partial_3 / n
    avg_bertscore_3 = total_bertscore_3 / n
    avg_hallucinated_3 = hallucinated_3 / n

    # Exibindo as métricas finais
    print("\nMÉTRICAS FINAIS - Modelo 3:")
    print(f"Bleu:                {avg_bleu_3:.4f}")
    print(f"Rouge:               {avg_rouge_3:.4f}")
    print(f"Coverage médio:      {avg_coverage_3:.2f}")
    print(f"Partial Match taxa:  {avg_partial_3:.2f}")
    print(f"BERTScore médio:     {avg_bertscore_3:.4f}")
    print(f"Alucinação:          {avg_hallucinated_3:.2f}")

    # Avaliando o modelo com base nas métricas
    if avg_bleu_3 >= bleu_threshold and avg_rouge_3 >= rouge_threshold and avg_coverage_3 >= coverage_threshold and \
       avg_partial_3 >= partial_threshold and avg_bertscore_3 >= bertscore_threshold and avg_hallucinated_3 <= hallucination_threshold:
        print("\n🔍 O modelo é bom!")
    else:
        print("\n⚠️ O modelo precisa de melhorias.")

#Chamada da função para avaliar o modelo
avaliar_modelo_3(total_bleu_3, total_rouge_3, total_coverage_3, total_partial_3, total_bertscore_3, hallucinated_3, n)


MÉTRICAS FINAIS - Modelo 3:
Bleu:                0.5447
Rouge:               0.5028
Coverage médio:      0.91
Partial Match taxa:  1.00
BERTScore médio:     0.9709
Alucinação:          0.67

🔍 O modelo é bom!


#MODELO 4

Alterações realizadas no Modelo Inicial:
*   Alteração do Threshold de 0.7 para 0.3 do recuperador *def retrieve_with_threshold_fn(pergunta, threshold)*

##Criação da cadeia RAG com Runnables LangChain

###PromptTemplate

In [763]:
qa_prompt_4 = PromptTemplate.from_template(
"""
Você é um assistente especialista em dados de planilhas. Com base no {context} do Excel, responda com precisão à pergunta: {question}
Responda de forma clara, objetiva e com linguagem técnica acessível. Se possível, cite a aba e a linha do Excel de onde vem a informação.
"""
)

###Cadeia de documentos com Runnable Passthrough

In [764]:
# Juntar documentos em um só contexto
document_chain_4 = RunnablePassthrough() | qa_prompt_4 | llm

###Recuperador

In [765]:
# Buscar documentos com score e filtrar por limiar
def retrieve_with_threshold_fn(pergunta, threshold=0.3):
    # Recupera documentos com seus scores
    resposta_obtida_4 = vectorstore.similarity_search_with_score(pergunta, k=10)
    # Filtra por similaridade mínima
    filtered_docs_4 = [doc for doc, score in resposta_obtida_4 if score >= threshold]
    # Se nada passou do limiar, retorna os top-3 independente do score
    if not filtered_docs_4:
       # print("⚠️ Nenhum documento acima do limiar. Usando os 3 mais similares disponíveis.")
        filtered_docs_4 = [doc for doc, _ in resposta_obtida_4[:3]]

    return filtered_docs_4[:5]  # Retorna os 5 melhores após filtro

###Fluxo de RAG usando RunnablePassthrought

In [766]:
# Função inteligente para juntar contexto (para debug, se quiser)
def juntar_contexto(docs):
    if not docs:
        raise ValueError("⚠️ Nenhum contexto relevante foi encontrado para essa pergunta.")

    first = docs[0]
    if isinstance(first, str):
        return "\n\n".join(docs)
    elif hasattr(first, "page_content"):
        return "\n\n".join(doc.page_content for doc in docs)
    else:
        raise ValueError(f"❌ Formato desconhecido: {type(first)}")

In [767]:
# Criação da retrieval_chain final: Faz a pergunta ➔ busca documentos ➔ junta contexto ➔ consulta o modelo.
retrieval_chain_4 = RunnablePassthrough().assign(
    question=lambda x: x["question"]
).assign(
    documents=lambda inputs: retrieve_with_threshold_fn(inputs["question"], threshold=0.3)
).assign(
    context=lambda inputs: juntar_contexto(inputs["documents"])
).assign(
    answer=lambda inputs: document_chain_4.invoke({
        "context": inputs["context"],
        "question": inputs["question"]
    })
)

###Chatbot interativo

In [768]:
#Função RAG que traz as respostas obtidas pelo modelo:
def rag_with_context(pergunta: str):
    max_retries = 3
    attempt = 0

    while attempt < max_retries:
        try:
            resposta_obtida_4 = retrieval_chain_4.invoke({"question": pergunta}) #LangChain que recupera e gera texto.

            # Tratamento das respostas obtidas e conteúdo
            if isinstance(resposta_obtida_4, dict):
                conteudo_4 = resposta_obtida_4.get("answer", resposta_obtida_4.get("output", resposta_obtida_4))
            else:
                conteudo_4 = resposta_obtida_4

            if hasattr(conteudo_4, "content"):
                resposta_texto_4 = conteudo_4.content
            elif isinstance(conteudo_4, str):
                resposta_texto_4 = conteudo_4
            else:
                resposta_texto_4 = str(conteudo_4)

            # Contexto
            contexto_usado_4 = resposta_obtida_4.get("context", "⚠️ Contexto não encontrado")
            return resposta_texto_4, contexto_usado_4

#resposta_obtida = resposta original, que pode vir em vários formatos.
#resposta_texto = texto limpo e padronizado, em formato strm pronto para ser retornado.

        except RateLimitError:
            attempt += 1
            if attempt < max_retries:
                print(f"🚫 Limite de requisições atingido. Tentando novamente em 10 segundos... (tentativa {attempt}/{max_retries})")
                time.sleep(10)
            else:
                print("❌ Limite de tentativas atingido. Tente novamente mais tarde.")
                return "", ""
        except Exception as e:
            print(f"⚠️ Erro inesperado: {e}")
            return "", ""

In [769]:
#Loop de perguntas

print("\n✅ Planilha carregada! Faça suas perguntas sobre os dados.")

while True:      #Para inserir perguntas manualmente
    query = input("\n❓ Digite sua pergunta ou 'sair': ")
    if query.lower() in ["sair", "exit", "quit"]:
        print("👋 Encerrando...")
        break

    resposta_texto_4, contexto_usado_4 = rag_with_context(query)

    if resposta_texto_4:
        print(f"\n 🤖 Resposta do bot:\n{resposta_texto_4}")


✅ Planilha carregada! Faça suas perguntas sobre os dados.

❓ Digite sua pergunta ou 'sair': Dê um exemplo de lipídio ionizável

 🤖 Resposta do bot:
Um exemplo de lipídio ionizável é o SM-102, que é utilizado em nanopartículas lipídicas para a entrega de RNA. Ele possui uma carga relativamente neutra em pH fisiológico, mas se torna positivamente carregado dentro da nanopartícula, permitindo a ligação com o backbone negativamente carregado do mRNA. Essa informação pode ser encontrada na [Planilha2], na linha 1.

❓ Digite sua pergunta ou 'sair': Em qual vacina o SM-102 é usado

 🤖 Resposta do bot:
O SM-102 é usado na vacina ARCT-154, desenvolvida pela Arcturus Therapeutics. Essa informação pode ser encontrada na [Planilha2], na linha 4.

❓ Digite sua pergunta ou 'sair': Qual vacina contém N1-metilpseudourina

 🤖 Resposta do bot:
A vacina que contém N1-Metilpseudourina é a vacina tozinameran (Pfizer-BioNTech) e elasomeran (Moderna). Essas informações podem ser encontradas na [Planilha2], 

##Métricas da qualidade das respostas e do modelo

#### a) Aplicação das métricas

In [965]:
#Normalização do texto:
def normalize(resposta_texto):
    resposta_texto = resposta_texto.lower()  # Deixa tudo minúsculo
    resposta_texto = re.sub(r'[^\w\s]', '', resposta_texto)  # Remove pontuação
    resposta_texto = re.sub(r'\s+', ' ', resposta_texto) # Substitui múltiplos espaços por um único espaço
    return resposta_texto.strip()  # Remove espaços nas bordas

In [966]:
def extract_entities(resposta_texto):
    doc = nlp(resposta_texto)
    return {ent.text.lower() for ent in doc.ents}

In [967]:
#BLEU (Bilingual Evaluation Understudy)
#Mede precisão = quantos n-gramas da resposta gerada aparecem na referência.
def calc_bleu(resposta_texto, resposta_esperada):
    smooth = SmoothingFunction().method4
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        ref_tokens = [ref.strip().split()]  # BLEU espera lista de listas
        pred_tokens = pred.strip().split()
        score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smooth)
        scores.append(score)

    return np.mean(scores)

In [968]:
# ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
#Mede recall = quanto da referência aparece na resposta
def calc_rouge(resposta_texto, resposta_esperada):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        score = scorer.score(ref.strip(), pred.strip())
        scores.append(score['rougeL'].fmeasure)

    return np.mean(scores)

In [969]:
# Coverage Score (quanto da resposta esperada estava na resposta gerada)
def calc_coverage(resposta_texto, resposta_esperada):
    ref_tokens = normalize(resposta_esperada).split()
    pred_tokens = normalize(resposta_texto).split()
    common = set(ref_tokens) & set(pred_tokens)
    if not common:
        return 0.0
    return len(common) / len(ref_tokens)

In [970]:
# Partial Match (modelo aceito se cobrir pelo menos 50% da resposta)
def calc_partial_match(resposta_texto, resposta_esperada, threshold=0.5):
    coverage = calc_coverage(resposta_texto, resposta_esperada)
    return int(coverage >= threshold)

In [971]:
#Taxa de alucinação = Percentual de respostas inventadas ou que fugiram do contexto
def check_hallucination(resposta_texto, contexto_usado):
    if "nenhuma informação suficiente" in resposta_texto_3.lower():
        return False  # isso NÃO é alucinação, é fallback correto

    resposta_entidades = extract_entities(resposta_texto)
    contexto_entidades = extract_entities(contexto_usado)

    # Verifica se todas as entidades na resposta estão presentes no contexto
    return not resposta_entidades.issubset(contexto_entidades)

In [972]:
def calc_bertscore(resposta_texto, resposta_esperada):
    P, R, F1 = score([resposta_texto], [resposta_esperada], lang='pt', verbose=False)
    return F1.item()

####b) Avaliação das respostas obtidas

In [973]:
# Conjunto de perguntas e respostas esperadas
avaliacao = [
     {"pergunta": "Dê um exemplo de lipídio ionizável", "resposta_esperada": "Um exemplo de lipídio ionizável é o SM-102"},
     {"pergunta": "Em qual vacina o SM-102 é usado", "resposta_esperada":"O SM-102 é usado na vacina da Moderna COVID-19"},
     {"pergunta": "Qual vacina contém N1-metilpseudourina", "resposta_esperada": "A vacina que contém N1-metilpseudourina é a tozinameran (Pfizer-BioNTech) e a elasomeran (Moderna)"},
     {"pergunta": "Qual a função da LNP?", "resposta_esperada": "LNP é uma estrutura nano baseada em lipídios que atua como veículo de entrega para RNA. Ela protege o RNA da degradação e facilita sua entrega nas células."},
     {"pergunta": "Como é formada uma LNP?", "resposta_esperada": "A LNP é formada por lipídios ionizáveis, fosfolipídios, colesterol e revestida por lipídios peguilados"},
     {"pergunta": "Dê exemplo de vacina com saRNA", "resposta_esperada": "Um exemplo de vacina com saRNA é Gemcovac, autorizada na Índia em 2022"}
]

In [974]:
print("\n🔍 Avaliação das Respostas:")
print("=" * 80)

#Inicialização das métricas globais
total_bleu_4, total_rouge_4, total_coverage_4, total_partial_4, total_bertscore_4 = 0, 0, 0, 0, 0
hallucinated_4 = 0

#Avaliação automática de respostas = compara as respostas obtidas x esperadas
for exemplo_4 in avaliacao:
    pergunta = exemplo_4["pergunta"]
    resposta_esperada_4 = exemplo_4["resposta_esperada"]
    try:
        resposta_texto_4, contexto_usado_4 = rag_with_context(pergunta)
    except Exception as e:
        print(f"❌ Erro ao buscar resposta para '{pergunta}': {e}")
        resposta_texto_4, contexto_usado_4 = "", ""

#Calcula as métricas da resposta
    bleu = calc_bleu(resposta_texto_4, resposta_esperada_4)
    rouge = calc_rouge(resposta_texto_4, resposta_esperada_4)
    coverage = calc_coverage(resposta_texto_4, resposta_esperada_4)
    partial = calc_partial_match(resposta_texto_4, resposta_esperada_4, threshold=0.5)
    aluc = check_hallucination(resposta_texto_4, contexto_usado_4)
    bertscore = calc_bertscore(resposta_texto_4, resposta_esperada_4)

#Acumula os resultados nas variáveis globais
    total_bleu_4 += bleu
    total_rouge_4 += rouge
    total_coverage_4 += coverage
    total_partial_4 += partial
    total_bertscore_4 += bertscore
    hallucinated_4 += int(aluc)

# Exibe avaliação por pergunta
    print(f"Pergunta: {pergunta}")
    print(f"Resposta esperada: {resposta_esperada_4}")
    print(f"Resposta do modelo: {resposta_texto_4}")
    print(f"Contexto: {contexto_usado_4[:200]}...")  # Mostra só o início
    print(f"bleu={bleu:.4f} | rouge={rouge:.4f} | Coverage={coverage:.2f} | Partial={partial} | BERTScore={bertscore:.4f} | Alucinação={'Sim' if aluc else 'Não'}")
    print("-" * 80)


🔍 Avaliação das Respostas:
Pergunta: Dê um exemplo de lipídio ionizável
Resposta esperada: Um exemplo de lipídio ionizável é o SM-102
Resposta do modelo: Um exemplo de lipídio ionizável é o SM-102.
Contexto: [Planilha2] Ionisable lipids like SM-102 hold a relatively (/ close to) neutral charge at physiological pH but are positively charged within the nanoparticle (the amine group is protonated to form an ...
bleu=0.8333 | rouge=0.7381 | Coverage=1.00 | Partial=1 | BERTScore=0.9823 | Alucinação=Não
--------------------------------------------------------------------------------
Pergunta: Em qual vacina o SM-102 é usado
Resposta esperada: O SM-102 é usado na vacina da Moderna COVID-19
Resposta do modelo: O SM-102 é usado na vacina da Moderna contra a COVID-19.
Contexto: [Planilha2] SM-102 is a synthetic amino lipid which is used in combination with other lipids to form lipid nanoparticles.[1] These are used for the delivery of mRNA-based vaccines,[2][3][4] and in par...
bleu=0.6522 | ro

#### b) Avaliação do modelo

#####***CRITÉRIOS***
#####**Bleu:** O modelo é considerado bom se o Bleu for superior a 0.3.
#####**Rouge:** O modelo é considerado bom se o Rouge for superior a 0.3.
#####**Coverage:** O modelo é considerado bom se o Coverage médio for superior a 0.7.
#####**Partial Match:** O modelo é considerado bom se a taxa de Partial Match for superior a 0.5.
#####**BERTScore:** O modelo é considerado bom se o BERTScore for superior a 0.8.
#####**Alucinação:** O modelo é considerado bom se a taxa de alucinação for menor que 0.7.

#####***AVALIAÇÃO***

In [976]:
n = len(avaliacao)

# Função e critérios para avaliar se o modelo é bom, com base nas métricas finais
def avaliar_modelo_4(total_bleu_4, total_rouge_4, total_coverage_4, total_partial_4, total_bertscore_4, hallucinated_4, n):
    bleu_threshold = 0.3
    rouge_threshold = 0.3
    coverage_threshold = 0.7
    partial_threshold = 0.5
    bertscore_threshold = 0.8
    hallucination_threshold = 0.7

    # Calculando as métricas médias
    avg_bleu_4 = total_bleu_4 / n
    avg_rouge_4 = total_rouge_4 / n
    avg_coverage_4 = total_coverage_4 / n
    avg_partial_4 = total_partial_4 / n
    avg_bertscore_4 = total_bertscore_4 / n
    avg_hallucinated_4 = hallucinated_4 / n

    # Exibindo as métricas finais
    print("\nMÉTRICAS FINAIS - Modelo 4:")
    print(f"Bleu:                {avg_bleu_4:.4f}")
    print(f"Rouge:               {avg_rouge_4:.4f}")
    print(f"Coverage médio:      {avg_coverage_4:.2f}")
    print(f"Partial Match taxa:  {avg_partial_4:.2f}")
    print(f"BERTScore médio:     {avg_bertscore_4:.4f}")
    print(f"Alucinação:          {avg_hallucinated_4:.2f}")

    # Avaliando o modelo com base nas métricas
    if avg_bleu_4 >= bleu_threshold and avg_rouge_4 >= rouge_threshold and avg_coverage_4 >= coverage_threshold and \
       avg_partial_4 >= partial_threshold and avg_bertscore_4 >= bertscore_threshold and avg_hallucinated_4 <= hallucination_threshold:
        print("\n🔍 O modelo é bom!")
    else:
        print("\n⚠️ O modelo precisa de melhorias.")

#Chamada da função para avaliar o modelo
avaliar_modelo_4(total_bleu_4, total_rouge_4, total_coverage_4, total_partial_4, total_bertscore_4, hallucinated_4, n)


MÉTRICAS FINAIS - Modelo 4:
Bleu:                0.4111
Rouge:               0.3841
Coverage médio:      0.80
Partial Match taxa:  1.00
BERTScore médio:     0.9341
Alucinação:          0.67

🔍 O modelo é bom!


#MODELO 5

Alterações realizadas no Modelo Inicial:
* Alteração no prompt
*   Alteração do Threshold de 0.7 para 0.9 do recuperador *def retrieve_with_threshold_fn(pergunta, threshold)*
* Alteração do K de 10 para 5 do recuperador *def retrieve_with_threshold_fn(pergunta, threshold)*

##Criação da cadeia RAG com Runnables LangChain

###PromptTemplate

In [977]:
qa_prompt_5 = PromptTemplate.from_template(
"""
Você é um assistente especialista em dados de planilhas. Com base no {context} do Excel, responda com precisão à pergunta: {question}

Responder sempre em português do Brasil;
Responder com até 20 palavras;
NÃO inventar respostas.
Seja direto e técnico.
Se a resposta não estiver clara no contexto, dizer: "Informação não encontrada no contexto."
"""
)

###Cadeia de documentos com Runnable Passthrough

In [978]:
# Juntar documentos em um só contexto
document_chain_5 = RunnablePassthrough() | qa_prompt_5 | llm

###Recuperador

In [980]:
# Buscar documentos com score e filtrar por limiar
def retrieve_with_threshold_fn(pergunta, threshold=0.9):
    # Recupera documentos com seus scores
    resposta_obtida_5 = vectorstore.similarity_search_with_score(pergunta, k=5)
    # Filtra por similaridade mínima
    filtered_docs_5 = [doc for doc, score in resposta_obtida_5 if score >= threshold]
    # Se nada passou do limiar, retorna os top-3 independente do score
    if not filtered_docs_5:
       # print("⚠️ Nenhum documento acima do limiar. Usando os 3 mais similares disponíveis.")
        filtered_docs_5 = [doc for doc, _ in resposta_obtida_5[:3]]

    return filtered_docs_5[:5]  # Retorna os 5 melhores após filtro

###Fluxo de RAG usando RunnablePassthrought

In [981]:
# Função inteligente para juntar contexto (para debug, se quiser)
def juntar_contexto(docs):
    if not docs:
        raise ValueError("⚠️ Nenhum contexto relevante foi encontrado para essa pergunta.")

    first = docs[0]
    if isinstance(first, str):
        return "\n\n".join(docs)
    elif hasattr(first, "page_content"):
        return "\n\n".join(doc.page_content for doc in docs)
    else:
        raise ValueError(f"❌ Formato desconhecido: {type(first)}")

In [982]:
# Criação da retrieval_chain final: Faz a pergunta ➔ busca documentos ➔ junta contexto ➔ consulta o modelo.
retrieval_chain_5 = RunnablePassthrough().assign(
    question=lambda x: x["question"]
).assign(
    documents=lambda inputs: retrieve_with_threshold_fn(inputs["question"], threshold=0.9)
).assign(
    context=lambda inputs: juntar_contexto(inputs["documents"])
).assign(
    answer=lambda inputs: document_chain_5.invoke({
        "context": inputs["context"],
        "question": inputs["question"]
    })
)

###Chatbot interativo

In [983]:
#Função RAG que traz as respostas obtidas pelo modelo:
def rag_with_context(pergunta: str):
    max_retries = 3
    attempt = 0

    while attempt < max_retries:
        try:
            resposta_obtida_5 = retrieval_chain_5.invoke({"question": pergunta}) #LangChain que recupera e gera texto.

            # Tratamento das respostas obtidas e conteúdo
            if isinstance(resposta_obtida_5, dict):
                conteudo_5 = resposta_obtida_5.get("answer", resposta_obtida_5.get("output", resposta_obtida_5))
            else:
                conteudo_5 = resposta_obtida_5

            if hasattr(conteudo_5, "content"):
                resposta_texto_5 = conteudo_5.content
            elif isinstance(conteudo_5, str):
                resposta_texto_5 = conteudo_5
            else:
                resposta_texto_5 = str(conteudo_5)

            # Contexto
            contexto_usado_5 = resposta_obtida_5.get("context", "⚠️ Contexto não encontrado")
            return resposta_texto_5, contexto_usado_5

#resposta_obtida = resposta original, que pode vir em vários formatos.
#resposta_texto = texto limpo e padronizado, em formato strm pronto para ser retornado.

        except RateLimitError:
            attempt += 1
            if attempt < max_retries:
                print(f"🚫 Limite de requisições atingido. Tentando novamente em 10 segundos... (tentativa {attempt}/{max_retries})")
                time.sleep(10)
            else:
                print("❌ Limite de tentativas atingido. Tente novamente mais tarde.")
                return "", ""
        except Exception as e:
            print(f"⚠️ Erro inesperado: {e}")
            return "", ""

In [984]:
#Loop de perguntas

print("\n✅ Planilha carregada! Faça suas perguntas sobre os dados.")

while True:      #Para inserir perguntas manualmente
    query = input("\n❓ Digite sua pergunta ou 'sair': ")
    if query.lower() in ["sair", "exit", "quit"]:
        print("👋 Encerrando...")
        break

    resposta_texto_5, contexto_usado_5 = rag_with_context(query)

    if resposta_texto_5:
        print(f"\n 🤖 Resposta do bot:\n{resposta_texto_5}")


✅ Planilha carregada! Faça suas perguntas sobre os dados.

❓ Digite sua pergunta ou 'sair': Dê um exemplo de lipídio ionizável

 🤖 Resposta do bot:
Um exemplo de lipídio ionizável é o SM-102.

❓ Digite sua pergunta ou 'sair': Em qual vacina o SM-102 é usado

 🤖 Resposta do bot:
O SM-102 é usado na vacina da Moderna contra a COVID-19.

❓ Digite sua pergunta ou 'sair': Qual vacina contém N1-metilpseudourina

 🤖 Resposta do bot:
A vacina que contém N1-metilpseudourina é a tozinameran (Pfizer-BioNTech).

❓ Digite sua pergunta ou 'sair': Qual a função da LNP

 🤖 Resposta do bot:
A função da LNP é atuar como veículo de entrega para o RNA, protegendo-o da degradação e facilitando sua entrada nas células.

❓ Digite sua pergunta ou 'sair': Como é formada uma LNP

 🤖 Resposta do bot:
Uma LNP é formada por lipídios ionizáveis, fosfolipídios, colesterol e é revestida por lipídios peguilados.

❓ Digite sua pergunta ou 'sair': Dê exemplo de vacina com saRNA

 🤖 Resposta do bot:
Exemplo de vacina co

##Métricas da qualidade das respostas e do modelo

#### a) Aplicação das métricas

In [998]:
#Normalização do texto:
def normalize(resposta_texto):
    resposta_texto = resposta_texto.lower()  # Deixa tudo minúsculo
    resposta_texto = re.sub(r'[^\w\s]', '', resposta_texto)  # Remove pontuação
    resposta_texto = re.sub(r'\s+', ' ', resposta_texto) # Substitui múltiplos espaços por um único espaço
    return resposta_texto.strip()  # Remove espaços nas bordas

In [999]:
def extract_entities(resposta_texto):
    doc = nlp(resposta_texto)
    return {ent.text.lower() for ent in doc.ents}

In [1000]:
#BLEU (Bilingual Evaluation Understudy)
#Mede precisão = quantos n-gramas da resposta gerada aparecem na referência.
def calc_bleu(resposta_texto, resposta_esperada):
    smooth = SmoothingFunction().method4
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        ref_tokens = [ref.strip().split()]  # BLEU espera lista de listas
        pred_tokens = pred.strip().split()
        score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smooth)
        scores.append(score)

    return np.mean(scores)

In [1001]:
# ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
#Mede recall = quanto da referência aparece na resposta
def calc_rouge(resposta_texto, resposta_esperada):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = []

    for pred, ref in zip(resposta_texto, resposta_esperada):
        score = scorer.score(ref.strip(), pred.strip())
        scores.append(score['rougeL'].fmeasure)

    return np.mean(scores)

In [1002]:
# Coverage Score (quanto da resposta esperada estava na resposta gerada)
def calc_coverage(resposta_texto, resposta_esperada):
    ref_tokens = normalize(resposta_esperada).split()
    pred_tokens = normalize(resposta_texto).split()
    common = set(ref_tokens) & set(pred_tokens)
    if not common:
        return 0.0
    return len(common) / len(ref_tokens)

In [1003]:
# Partial Match (modelo aceito se cobrir pelo menos 50% da resposta)
def calc_partial_match(resposta_texto, resposta_esperada, threshold=0.5):
    coverage = calc_coverage(resposta_texto, resposta_esperada)
    return int(coverage >= threshold)

In [1004]:
#Taxa de alucinação = Percentual de respostas inventadas ou que fugiram do contexto
def check_hallucination(resposta_texto, contexto_usado):
    if "nenhuma informação suficiente" in resposta_texto_3.lower():
        return False  # isso NÃO é alucinação, é fallback correto

    resposta_entidades = extract_entities(resposta_texto)
    contexto_entidades = extract_entities(contexto_usado)

    # Verifica se todas as entidades na resposta estão presentes no contexto
    return not resposta_entidades.issubset(contexto_entidades)

In [1005]:
def calc_bertscore(resposta_texto, resposta_esperada):
    P, R, F1 = score([resposta_texto], [resposta_esperada], lang='pt', verbose=False)
    return F1.item()

####b) Avaliação das respostas obtidas

In [1006]:
# Conjunto de perguntas e respostas esperadas
avaliacao = [
     {"pergunta": "Dê um exemplo de lipídio ionizável", "resposta_esperada": "Um exemplo de lipídio ionizável é o SM-102"},
     {"pergunta": "Em qual vacina o SM-102 é usado", "resposta_esperada":"O SM-102 é usado na vacina da Moderna COVID-19"},
     {"pergunta": "Qual vacina contém N1-metilpseudourina", "resposta_esperada": "A vacina que contém N1-metilpseudourina é a tozinameran (Pfizer-BioNTech) e a elasomeran (Moderna)"},
     {"pergunta": "Qual a função da LNP?", "resposta_esperada": "LNP é uma estrutura nano baseada em lipídios que atua como veículo de entrega para RNA. Ela protege o RNA da degradação e facilita sua entrega nas células."},
     {"pergunta": "Como é formada uma LNP?", "resposta_esperada": "A LNP é formada por lipídios ionizáveis, fosfolipídios, colesterol e revestida por lipídios peguilados"},
     {"pergunta": "Dê exemplo de vacina com saRNA", "resposta_esperada": "Um exemplo de vacina com saRNA é Gemcovac, autorizada na Índia em 2022"}
]

In [1007]:
print("\n🔍 Avaliação das Respostas:")
print("=" * 80)

#Inicialização das métricas globais
total_bleu_5, total_rouge_5, total_coverage_5, total_partial_5, total_bertscore_5 = 0, 0, 0, 0, 0
hallucinated_5 = 0

#Avaliação automática de respostas = compara as respostas obtidas x esperadas
for exemplo_5 in avaliacao:
    pergunta = exemplo_5["pergunta"]
    resposta_esperada_5 = exemplo_5["resposta_esperada"]
    try:
        resposta_texto_5, contexto_usado_5 = rag_with_context(pergunta)
    except Exception as e:
        print(f"❌ Erro ao buscar resposta para '{pergunta}': {e}")
        resposta_texto_5, contexto_usado_5 = "", ""

#Calcula as métricas da resposta
    bleu = calc_bleu(resposta_texto_5, resposta_esperada_5)
    rouge = calc_rouge(resposta_texto_5, resposta_esperada_5)
    coverage = calc_coverage(resposta_texto_5, resposta_esperada_5)
    partial = calc_partial_match(resposta_texto_5, resposta_esperada_5, threshold=0.5)
    aluc = check_hallucination(resposta_texto_5, contexto_usado_5)
    bertscore = calc_bertscore(resposta_texto_5, resposta_esperada_5)

#Acumula os resultados nas variáveis globais
    total_bleu_5 += bleu
    total_rouge_5 += rouge
    total_coverage_5 += coverage
    total_partial_5 += partial
    total_bertscore_5 += bertscore
    hallucinated_5 += int(aluc)

# Exibe avaliação por pergunta
    print(f"Pergunta: {pergunta}")
    print(f"Resposta esperada: {resposta_esperada_5}")
    print(f"Resposta do modelo: {resposta_texto_5}")
    print(f"Contexto: {contexto_usado_5[:200]}...")  # Mostra só o início
    print(f"bleu={bleu:.4f} | rouge={rouge:.4f} | Coverage={coverage:.2f} | Partial={partial} | BERTScore={bertscore:.4f} | Alucinação={'Sim' if aluc else 'Não'}")
    print("-" * 80)


🔍 Avaliação das Respostas:
Pergunta: Dê um exemplo de lipídio ionizável
Resposta esperada: Um exemplo de lipídio ionizável é o SM-102
Resposta do modelo: Um exemplo de lipídio ionizável é o SM-102.
Contexto: [Planilha2] Ionisable lipids like SM-102 hold a relatively (/ close to) neutral charge at physiological pH but are positively charged within the nanoparticle (the amine group is protonated to form an ...
bleu=0.8333 | rouge=0.7381 | Coverage=1.00 | Partial=1 | BERTScore=0.9823 | Alucinação=Não
--------------------------------------------------------------------------------
Pergunta: Em qual vacina o SM-102 é usado
Resposta esperada: O SM-102 é usado na vacina da Moderna COVID-19
Resposta do modelo: O SM-102 é usado na vacina da Moderna contra a COVID-19.
Contexto: [Planilha2] SM-102 is a synthetic amino lipid which is used in combination with other lipids to form lipid nanoparticles.[1] These are used for the delivery of mRNA-based vaccines,[2][3][4] and in par...
bleu=0.6522 | ro

#### b) Avaliação do modelo

#####***CRITÉRIOS***
#####**Bleu:** O modelo é considerado bom se o Bleu for superior a 0.3.
#####**Rouge:** O modelo é considerado bom se o Rouge for superior a 0.3.
#####**Coverage:** O modelo é considerado bom se o Coverage médio for superior a 0.7.
#####**Partial Match:** O modelo é considerado bom se a taxa de Partial Match for superior a 0.5.
#####**BERTScore:** O modelo é considerado bom se o BERTScore for superior a 0.8.
#####**Alucinação:** O modelo é considerado bom se a taxa de alucinação for menor que 0.7.

#####***AVALIAÇÃO***

In [1008]:
n = len(avaliacao)

# Função e critérios para avaliar se o modelo é bom, com base nas métricas finais
def avaliar_modelo_5(total_bleu_5, total_rouge_5, total_coverage_5, total_partial_5, total_bertscore_5, hallucinated_5, n):
    bleu_threshold = 0.3
    rouge_threshold = 0.3
    coverage_threshold = 0.7
    partial_threshold = 0.5
    bertscore_threshold = 0.8
    hallucination_threshold = 0.7

    # Calculando as métricas médias
    avg_bleu_5 = total_bleu_5 / n
    avg_rouge_5 = total_rouge_5 / n
    avg_coverage_5 = total_coverage_5 / n
    avg_partial_5 = total_partial_5 / n
    avg_bertscore_5 = total_bertscore_5 / n
    avg_hallucinated_5 = hallucinated_5 / n

    # Exibindo as métricas finais
    print("\nMÉTRICAS FINAIS - Modelo 5:")
    print(f"Bleu:                {avg_bleu_5:.4f}")
    print(f"Rouge:               {avg_rouge_5:.4f}")
    print(f"Coverage médio:      {avg_coverage_5:.2f}")
    print(f"Partial Match taxa:  {avg_partial_5:.2f}")
    print(f"BERTScore médio:     {avg_bertscore_5:.4f}")
    print(f"Alucinação:          {avg_hallucinated_5:.2f}")

    # Avaliando o modelo com base nas métricas
    if avg_bleu_5 >= bleu_threshold and avg_rouge_5 >= rouge_threshold and avg_coverage_5 >= coverage_threshold and \
       avg_partial_5 >= partial_threshold and avg_bertscore_5 >= bertscore_threshold and avg_hallucinated_5 <= hallucination_threshold:
        print("\n🔍 O modelo é bom!")
    else:
        print("\n⚠️ O modelo precisa de melhorias.")

#Chamada da função para avaliar o modelo
avaliar_modelo_5(total_bleu_5, total_rouge_5, total_coverage_5, total_partial_5, total_bertscore_5, hallucinated_5, n)



MÉTRICAS FINAIS - Modelo 5:
Bleu:                0.3208
Rouge:               0.3039
Coverage médio:      0.82
Partial Match taxa:  1.00
BERTScore médio:     0.9413
Alucinação:          0.67

🔍 O modelo é bom!
